<a href="https://colab.research.google.com/github/worldwidekatie/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [67]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-03-11 22:08:50--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.97.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.97.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  13.2MB/s    in 16s     

2020-03-11 22:09:07 (12.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [68]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [69]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [70]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

**First, write down which columns you need and which dataframes have them.**

Really I just need the overall product name count but to get there I need to make a larger table by concatenating prior and train and then I need to merge that with product id and name from the products table. Then I can do value counts for the top ten product names.


In [0]:
import pandas as pd

prior = pd.read_csv('order_products__prior.csv')

In [0]:
train = pd.read_csv('order_products__train.csv')

In [0]:
products = pd.read_csv('products.csv')

products = products[['product_id', 'product_name']]
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
prior = prior['product_id']

train = train['product_id']

df = pd.concat([prior, train])

df.head()

0    33120
1    28985
2     9327
3    45918
4    30035
Name: product_id, dtype: int64

**Next, merge these into a single dataframe.**

In [0]:
#merge df and products based on product id
final_df = pd.merge(df, products, on='product_id', how='inner')

final_df.head()

,product_id,product_name
0,33120,Organic Egg Whites
1,33120,Organic Egg Whites
2,33120,Organic Egg Whites
3,33120,Organic Egg Whites
4,33120,Organic Egg Whites


**Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products**

In [0]:
final_df['product_name'].value_counts().head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy
#Or as I've named them, table3 -> tidy2

#Start by flipping it back around with T
table3 = table2.T

table3

,treatmenta,treatmentb
John Smith,NaN,2.0
Jane Doe,16.0,11.0
Mary Johnson,3.0,1.0


In [0]:
#Then reset the index to put names in a real column
#So you can work with them later
table3 = table3.reset_index()

table3

,index,treatmenta,treatmentb
0,John Smith,NaN,2.0
1,Jane Doe,16.0,11.0
2,Mary Johnson,3.0,1.0


In [0]:
#Now that names are available to use for id_vars, use melt to melt together
#treatmenta and treatmentb and melt together the values that were in those
#two columns when they were seperate
tidy2 = table3.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])

tidy2

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
#Then rename them from the generic names like index and variable
#so it matches what our data actually shows
tidy2 = tidy2.rename(columns={
    'index': 'name',
    'variable': 'trt',
    'value': 'result'
})

tidy2

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
#Then take away treatment because we already know it's treatment
#because we've made that column trt and it looks better/saves room
#when it's just a and b.
tidy2.trt = tidy2.trt.str.replace('treatment', '')
tidy2

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [0]:
# tidy -> table2
#Or as I've named them, tidy2 -> table
table = tidy2.pivot_table(index='name', columns='trt', values='result')
table


trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


Create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
# Flights Pivot Table

import seaborn as sns
flights = sns.load_dataset('flights')

flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [0]:
flights.pivot_table(index='year', 
                    columns='month')

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
#Read in all the CSVs I need for this
train = pd.read_csv('order_products__train.csv')

In [0]:
products = pd.read_csv('products.csv')

In [0]:
prior = pd.read_csv('order_products__prior.csv')

In [0]:
orders = pd.read_csv('orders.csv')

In [80]:
#concate and slim the first two.
train = train[['product_id', 'order_id']]

prior = prior[['product_id', 'order_id']]

df1 = pd.concat([train, prior])

df1.head()


,product_id,order_id
0,49302,1
1,11109,1
2,10246,1
3,49683,1
4,43633,1


In [81]:
#Slim down the others
products = products[['product_id', 'product_name']]

products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [82]:
orders = orders[['order_id', 'order_hour_of_day']]

orders.head()

,order_id,order_hour_of_day
0,2539329,8
1,2398795,7
2,473747,12
3,2254736,7
4,431534,15


In [83]:
#Merge them
df2 = pd.merge(orders, df1)

df2.head()

,order_id,order_hour_of_day,product_id
0,2539329,8,196
1,2539329,8,14084
2,2539329,8,12427
3,2539329,8,26088
4,2539329,8,26405


In [84]:
finaldf = pd.merge(df2, products)

finaldf.head()

,order_id,order_hour_of_day,product_id,product_name
0,2539329,8,196,Soda
1,2398795,7,196,Soda
2,473747,12,196,Soda
3,2254736,7,196,Soda
4,431534,15,196,Soda


In [86]:
df = finaldf[['order_hour_of_day', 'product_name']]

df.head()

,order_hour_of_day,product_name
0,8,Soda
1,7,Soda
2,12,Soda
3,7,Soda
4,15,Soda


In [97]:
#Just to see if this way was working, I went with 17 which is where the late-night
#list is supposed to be most prevalent
condition = df['order_hour_of_day'] == 17

df_late = df[condition]

df_late.head()

df_late['order_hour_of_day'].value_counts()

17    2186409
Name: order_hour_of_day, dtype: int64

In [98]:
#It still does not work and that makes no sense to me. 
df_late['product_name'].value_counts().head(25)

Banana                        32019
Bag of Organic Bananas        24866
Organic Strawberries          17065
Organic Baby Spinach          16078
Organic Hass Avocado          14504
Organic Avocado               12226
Large Lemon                   10420
Limes                          9380
Strawberries                   9163
Organic Raspberries            8753
Organic Whole Milk             8391
Organic Garlic                 7203
Organic Yellow Onion           7168
Organic Zucchini               6841
Cucumber Kirby                 6666
Organic Blueberries            6495
Organic Fuji Apple             5878
Organic Lemon                  5854
Organic Grape Tomatoes         5677
Organic Cucumber               5537
Seedless Red Grapes            5244
Apple Honeycrisp Organic       5222
Honeycrisp Apple               5155
Yellow Onions                  5056
Sparkling Water Grapefruit     4997
Name: product_name, dtype: int64

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####